In [ ]:
##DAY-1 /EDA, Scaling....

import pandas as pd
import numpy as np
df= pd.read_csv('creditcard.csv')
#df['Class'].value_counts()
import seaborn as sb
import matplotlib.pyplot as plt
#sb.countplot(x='Class',data=df)
#plt.title('fraud vs legit')

# import matplotlib.pyplot as plt
# import seaborn as sns

# v_features = ['V1', 'V2', 'V3']

# plt.figure(figsize=(12, 6))

# for i, v in enumerate(v_features):
#     plt.subplot(1, 3, i+1)                 # 1 row, 3 cols, i-th plot
#     sns.histplot(df[v], bins=50, kde=True)
#     plt.title(f'Distribution of {v}')

# plt.tight_layout()
# plt.show()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


df_scaled=df.copy()
scaler=StandardScaler()
df_scaled[['Amount','Time']]= scaler.fit_transform(df[['Amount','Time']])

df_scaled[['Amount','Time']].describe()

X = df_scaled.drop('Class', axis=1)
y = df_scaled['Class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# print(X_train.shape)
# print(X_test.shape)
# print(df.shape)
corr = df.corr()
plt.figure(figsize=(12,8))
sb.heatmap(corr, cmap='coolwarm',annot=False)
plt.title('correlation heatmap')



In [ ]:
##Day-2/UNI-variate anomaly det
features = ['V17', 'V14', 'V12']
X_uni = X_train[features]
# print(X_uni.shape)
def estimate_gaussian(X):
    mu = X.mean(axis=0)
    sigma2 = X.var(axis=0)
    return mu, sigma2

mu, sigma2 = estimate_gaussian(X_uni)
#
from scipy.stats import norm

def calculate_probability(X, mu, sigma2):
    p = 1
    for i in range(len(mu)):
        p_i = norm.pdf(X.iloc[:, i], loc=mu.iloc[i], scale=np.sqrt(sigma2.iloc[i]))
        p *= p_i 
    return p

p_train = calculate_probability(X_uni, mu, sigma2)
#
from sklearn.metrics import f1_score, precision_score, recall_score

def select_threshold(y_val, p_val):
    best_epsilon = 0
    best_f1 = 0
    epsilons = np.linspace(min(p_val), max(p_val), 1000)

    for epsilon in epsilons:
        preds = (p_val < epsilon).astype(int)
        f1 = f1_score(y_val, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_epsilon = epsilon

    return best_epsilon, best_f1

X_val_uni = X_test[features]
p_val = calculate_probability(X_val_uni, mu, sigma2)

epsilon, best_f1 = select_threshold(y_test, p_val)
#
y_pred_uni = (p_val < epsilon).astype(int)

precision = precision_score(y_test, y_pred_uni)
recall = recall_score(y_test, y_pred_uni)
f1 = f1_score(y_test, y_pred_uni)

print(f"Univariate Gaussian → Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")




In [ ]:
##DAY-2/ Multi-variate Anomaly det
features = ['V14', 'V12', 'V10', 'V17', 'V16', 'V3', 'V7', 'V11', 'V18']
X_m = X_train[features]
X_val_m = X_test[features]
#
def estimate_gaussian_multi(X):
    mu = X.mean(axis=0)
    sigma = np.cov(X.T)  
    return mu, sigma

mu_m, sigma_m = estimate_gaussian_multi(X_m)
#
from scipy.stats import multivariate_normal

def calculate_multivariate_prob(X, mu, sigma):
    dist = multivariate_normal(mean=mu, cov=sigma)
    return dist.pdf(X)

p_val_m = calculate_multivariate_prob(X_val_m, mu_m, sigma_m)
#
epsilon_m, best_f1_m = select_threshold(y_test, p_val_m)
#
y_pred_m = (p_val_m < epsilon_m).astype(int)

precision_m = precision_score(y_test, y_pred_m)
recall_m = recall_score(y_test, y_pred_m)
f1_m = f1_score(y_test, y_pred_m)

print(f"Multivariate Gaussian → Precision: {precision_m:.4f}, Recall: {recall_m:.4f}, F1: {f1_m:.4f}")
#


In [ ]:
##DAY-3 / PCA & LogREG
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
#
from sklearn.model_selection import train_test_split

X_train_pca, X_test_pca, y_train_lr, y_test_lr = train_test_split(
    X_pca, y, test_size=0.2, stratify=y, random_state=42
)
#
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight='balanced',random_state=42)
lr.fit(X_train_pca, y_train_lr)
#
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

y_pred_lr = lr.predict(X_test_pca)
print(classification_report(y_test_lr, y_pred_lr, digits=4))
# #
# from sklearn.metrics import precision_recall_curve
# import matplotlib.pyplot as plt

# y_scores = lr.predict_proba(X_test_pca)[:,1] 
# y_pred_custom = (y_scores > 0.0001).astype(int)

# precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

# plt.plot(recall, precision)
# plt.title("Precision-Recall Curve - Logistic Regression (PCA)")
# plt.xlabel("Recall")
# plt.ylabel("Precision")
# plt.grid()
# plt.show()


In [ ]:
##Hybrid + evaluation framework

y_hybrid = ((y_pred_uni == 1) | (y_pred_m == 1) | (y_pred_lr == 1)).astype(int)
#
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def evaluate_model(y_true, y_pred, model_name="Model"):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\n {model_name}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)

#
evaluate_model(y_test, y_pred_uni, "Univariate Gaussian")
evaluate_model(y_test, y_pred_m, "Multivariate Gaussian")
evaluate_model(y_test, y_pred_lr, "Logistic Regression (PCA)")
evaluate_model(y_test, y_hybrid, "Hybrid Model (OR Vote)")



In [ ]:
##Summary cell

summary = []

# Univariate
summary.append([
    "Univariate Gaussian",
    precision_score(y_test, y_pred_uni),
    recall_score(y_test, y_pred_uni),
    f1_score(y_test, y_pred_uni)
])

# Multivariate
summary.append([
    "Multivariate Gaussian",
    precision_score(y_test, y_pred_m),
    recall_score(y_test, y_pred_m),
    f1_score(y_test, y_pred_m)
])

# Logistic Regression
summary.append([
    "Logistic Regression",
    precision_score(y_test, y_pred_lr),
    recall_score(y_test, y_pred_lr),
    f1_score(y_test, y_pred_lr)
])

# Hybrid Model
summary.append([
    "Hybrid Model",
    precision_score(y_test, y_hybrid),
    recall_score(y_test, y_hybrid),
    f1_score(y_test, y_hybrid)
])

df_summary = pd.DataFrame(summary, columns=["Model", "Precision", "Recall", "F1"])
df_summary = df_summary.round(4)

print(df_summary)

# Export to markdown (for README)
print("\nMarkdown Table:\n")
print(df_summary.to_markdown(index=False, tablefmt="github"))


